In [2]:
import os, time
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.14.0


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.expand_dims(x_train, -1) #(60000, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

# CPU

In [6]:
os.environ['COLAB_GPU']

'0'

In [0]:
start=time.time()
model = create_model()
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train[:64*60*8].astype(np.float32), y_train[:64*60*8].astype(np.float32), #fair comparison with TPU: batch of 64*8 (8 TPUs), each epoch has 60 steps
    epochs=10,
    batch_size=64,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=10
)

model.save_weights('./fashion_mnist.h5', overwrite=True)
print(f'{time.time()-start}s taken')

W0729 20:51:59.397288 140681657165696 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 30720 samples, validate on 10000 samples
Epoch 1/10
 5120/30720 [====>.........................] - ETA: 5:30 - loss: 1.4247 - sparse_categorical_accuracy: 0.6197

# GPU

In [6]:
os.environ['COLAB_GPU']

'1'

In [10]:
start=time.time()
model = create_model()
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train[:64*60*8].astype(np.float32), y_train[:64*60*8].astype(np.float32), #fair comparison with TPU: batch of 64*8 (8 TPUs), each epoch has 60 steps
    epochs=10,
    batch_size=64,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=10
)

model.save_weights('./fashion_mnist.h5', overwrite=True)
print(f'{time.time()-start}s taken')

Train on 30720 samples, validate on 10000 samples
Epoch 1/10
30720/30720 [==============================] - 6s 185us/sample - loss: 0.8160 - sparse_categorical_accuracy: 0.7441
Epoch 2/10
30720/30720 [==============================] - 5s 168us/sample - loss: 0.4839 - sparse_categorical_accuracy: 0.8329
Epoch 3/10
30720/30720 [==============================] - 5s 168us/sample - loss: 0.3931 - sparse_categorical_accuracy: 0.8598
Epoch 4/10
30720/30720 [==============================] - 5s 169us/sample - loss: 0.3353 - sparse_categorical_accuracy: 0.8799
Epoch 5/10
30720/30720 [==============================] - 5s 170us/sample - loss: 0.3077 - sparse_categorical_accuracy: 0.8867
Epoch 6/10
30720/30720 [==============================] - 5s 170us/sample - loss: 0.2860 - sparse_categorical_accuracy: 0.8941
Epoch 7/10
30720/30720 [==============================] - 5s 170us/sample - loss: 0.2702 - sparse_categorical_accuracy: 0.8995
Epoch 8/10
30720/30720 [==============================] - 5s 

# TPU

In [7]:
tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print ('TPU address is', tpu_address)

with tf.Session(tpu_address) as session:
  devices = session.list_devices()

print(f'{len(devices)} devices:')
for d in devices: print(d)

TPU address is grpc://10.34.100.2:8470
11 devices:
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9908414365363191594)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4974917026288295601)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7998588864879986827)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15565643528276174813)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17190374650284067929)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 7987109689500302738)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 2310587764381307383)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11327104696702171434)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 14475938025564265619)
_DeviceAtt

In [9]:
start=time.time()
resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    batch_size=512,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=10
)

model.save_weights('./fashion_mnist.h5', overwrite=True)
print(f'{time.time()-start}s taken')

W0729 20:48:08.765688 139820351559552 tpu_strategy_util.py:56] TPU system %s has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


Epoch 1/10
60/60 [==============================] - 4s 66ms/step - loss: 1.2361 - sparse_categorical_accuracy: 0.6615
Epoch 2/10
60/60 [==============================] - 1s 19ms/step - loss: 0.6103 - sparse_categorical_accuracy: 0.7977
Epoch 3/10
60/60 [==============================] - 1s 20ms/step - loss: 0.4897 - sparse_categorical_accuracy: 0.8312
Epoch 4/10
60/60 [==============================] - 1s 20ms/step - loss: 0.4528 - sparse_categorical_accuracy: 0.8438
Epoch 5/10
60/60 [==============================] - 1s 20ms/step - loss: 0.3996 - sparse_categorical_accuracy: 0.8593
Epoch 6/10
60/60 [==============================] - 1s 20ms/step - loss: 0.3726 - sparse_categorical_accuracy: 0.8687
Epoch 7/10
60/60 [==============================] - 1s 20ms/step - loss: 0.3469 - sparse_categorical_accuracy: 0.8769
Epoch 8/10
60/60 [==============================] - 1s 20ms/step - loss: 0.3225 - sparse_categorical_accuracy: 0.8838
Epoch 9/10
60/60 [==============================] - 1s 1